In [ ]:
from ftplib import FTP
import tempfile

from pathlib import Path
import numpy as np

from astropy.io import fits

from scipy.interpolate import interpn

#%matplotlib ipympl
%matplotlib inline
import matplotlib.pyplot as plt

import h5py

In [ ]:
# target data
Teff = 4970
logg = 3.32
FeH = 0.03
alphaM = 0.0

def get_interp_indices(val, lst):
    npl = np.array(lst)
    lo = np.searchsorted(npl, val, side='right', sorter = None)-1
    hi = np.searchsorted(npl, val, side='left', sorter = None)
    return lo, hi
    



In [ ]:
Teffs = list(range(2300, 7001, 100)) + list(range(7000, 12001, 200))
loggs = [ x/10 for x in range(0, 61, 5)]  # loggs (unnamed param in filename) [0.0 -> 6.0]
FeHs = [ -4.0, -3.0, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0] # Z [-4.0 -> +1.0]
alphaMs = [-0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2] # [-0.2 -> +1.2]

In [ ]:
alphaM_lohi = get_interp_indices(alphaM, alphaMs)
FeH_lohi = get_interp_indices(FeH, FeHs)
logg_lohi = get_interp_indices(logg, loggs)
Teff_lohi = get_interp_indices(Teff, Teffs)



In [ ]:
def makeftppath(Z, alpha, logg, teff):
    if alpha == 0.0: 
        if Z == 0.0:
            basedir = f"Z-{Z:1.1f}"
            filename = f"lte{teff:05d}-{logg:1.2f}-{Z:1.1f}.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits"
        else:
            basedir = f"Z{Z:+1.1f}"
            filename = f"lte{teff:05d}-{logg:1.2f}{Z:+1.1f}.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits"
    else:
        basedir = f"Z-{abs(Z):1.1f}.Alpha={alpha:+1.2f}"
        filename = f"lte{teff:05d}-{logg:1.2f}{Z:+1.1f}.Alpha={alpha:+1.2f}.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits"
    
    return basedir, filename

In [ ]:
files = [
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),
    
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    makeftppath(FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),

        ]

grid = [
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),    
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),

        ]


In [ ]:
def get_files(files, tempdir):
    with FTP("phoenix.astro.physik.uni-goettingen.de") as ftp:
        ftp.login()

        ftp.cwd("HiResFITS")
        #ftp.retrlines('LIST') 
        wavelength_grid = "WAVE_PHOENIX-ACES-AGSS-COND-2011.fits"
        with open(tempdir / wavelength_grid, "wb") as fp:
            ftp.retrbinary("RETR " + wavelength_grid, fp.write)
            
            
        datadir = "PHOENIX-ACES-AGSS-COND-2011"

        for d,f in files:
            ftp.cwd(f"{datadir}/"+d)
            with open(tempdir / f, "wb") as fp:
                ftp.retrbinary("RETR " + f, fp.write)
            ftp.cwd("../../")

In [ ]:
#with tempfile.TemporaryDirectory() as tmpdirname:
tmpdirname = tempfile.mkdtemp()

wavelength_grid = "WAVE_PHOENIX-ACES-AGSS-COND-2011.fits"
    

tempdir = Path(tmpdirname)
get_files(files, tempdir)     

In [ ]:
with fits.open(tempdir/wavelength_grid) as hdul:
    wavelengths = hdul[0].data

In [ ]:
spectra = []
for d, f in files:
    with fits.open(tempdir/files[0][1]) as hdul:
        spectra.append(hdul[0].data)

In [ ]:
grid = [
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[0]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),    
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[0]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[0]], Teffs[Teff_lohi[1]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[0]]),
    (FeHs[FeH_lohi[1]], alphaMs[alphaM_lohi[1]], loggs[logg_lohi[1]], Teffs[Teff_lohi[1]]),

        ]

In [ ]:
# FeH interpolation:
spectra2 = []

var_x = (FeHs[FeH_lohi[0]], FeHs[FeH_lohi[1]])
coord_x = FeH
rng = 8
for i in range(rng):
    if var_x[0] == var_x[1]:
        spectra2.append(spectra[i])
    else:
        spectra2.append((spectra[i+rng] - spectra[i])/(var_x[1] - var_x[0])*(coord_x - var_x[0])+spectra[i] )

In [ ]:
# alphaM interpolation:
spectra3 = []

var_x = (alphaMs[alphaM_lohi[0]], alphaMs[alphaM_lohi[1]])
coord_x = alphaM

rng = 4
for i in range(rng):
    if var_x[0] == var_x[1]:
        spectra3.append(spectra2[i])
    else:
        spectra3.append((spectra2[i+rng] - spectra2[i])/(var_x[1] - var_x[0])*(coord_x - var_x[0])+spectra2[i] )

In [ ]:
# logg interpolation:
spectra4 = []

var_x = (loggs[logg_lohi[0]], loggs[logg_lohi[1]])
coord_x = logg

rng = 2
for i in range(rng):
    if var_x[0] == var_x[1]:
        spectra4.append(spectra3[i])
    else:
        spectra4.append((spectra3[i+rng] - spectra3[i])/(var_x[1] - var_x[0])*(coord_x - var_x[0])+spectra3[i] )

In [ ]:
# Teff interpolation:
spectra5 = []

var_x = (Teffs[Teff_lohi[0]], Teffs[Teff_lohi[1]])
coord_x = Teff

rng = 1
for i in range(rng):
    if var_x[0] == var_x[1]:
        spectra5.append(spectra4[i])
    else:
        spectra5.append((spectra4[i+rng] - spectra4[i])/(var_x[1] - var_x[0])*(coord_x - var_x[0])+spectra4[i] )

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(14,10))

ax.plot(wavelengths, spectra5[0])



In [ ]:
with h5py.File("interpolated_stellar_spectrum.h5", "w") as f:
    lambda_dset = f.create_dataset("wavelength", (len(wavelengths),), dtype=np.float64)
    flux_dset = f.create_dataset("flux", spectra5[0].shape, dtype=np.float64)
    lambda_dset[...] = wavelengths
    flux_dset[...] = spectra5[0]